# **Superresolution and Enhancement**

Vinícius de Moraes - 13749910

Universidade de São Paulo, 2024, first semester

Asssignment 01 - SCC0251

## **Importing Libraries**

* **Imageio:** load images on the disk
* **Numpy:** manipulate matrices

In [ ]:
import imageio.v3 as imageio
import numpy as np

## **1. Introduction**

### **1.1 Goal**

To have the students implementing their first pixel and histogram-based enhancement methods as well as present the concept of superresolution for the first time.

### **1.2 Task**

In this assignment you have to implement 3 distinct image enhancement techniques, as well as a superresolution method based on multiple views of the same image.

1.   Find and load all low resolution images $l_i$ ∈ $L$ that match the basename imglow (i.e. filenames that start with imglow)
2.   Apply the selected enhancement method $F$ to all low resolution images, using parameter γ when appropriate
3.   Combine the low resolution images into a high resolution version $\hat{H}$
4.   Compare $\hat{H}$ against reference image $H$ using Root Mean Squared Error (RMSE)

### **1.3 Input Parameters**

The following parameters will be input to your program in the following order:

1.   Basename **imglow** for low resolution images $l_i$ ∈ $L$. The basename references the start of the filenames for 4 low resolution images $l_1$, $l_2$, $l_3$, $l_4$
2.   Filename **imghigh** for the high resolution image $H$
3.   Enhancement method identifier $F$ (0, 1, 2 or 3)
4.   Enhancement method parameter γ for $F$ = 3

In [ ]:
name_low_res = input().rstrip()
list_low_res = [name_low_res+str(i)+'.png' for i in range (0,4)]

07_low


In [ ]:
imglow = []

for i in range(0,4):
  imglow.append(imageio.imread(list_low_res[i]))

In [ ]:
name_high_res = input().rstrip()

07_high.png


In [ ]:
imghigh = imageio.imread(name_high_res)

In [ ]:
f = input().rstrip()

3


In [ ]:
gama = input().rstrip()

2.0


## **2. Image Enhancement**

There are three options for Image Enhancement, with Option 0 indicating that no enhancement is to be done:

**Option 0 - No Enhancement :** Do not apply any enhancement technique to the image and instead skip to the superresolution step.

**Options 1 and 2** are histogram-based methods while Option 3 uses pixel-based Gamma correction.

### **2.1 Histogram-based Enhancement**

In [ ]:
def histogram(img, no_levels):
  N, M = img.shape
  hist = np.zeros(no_levels).astype(int)

  for i in range(no_levels):
    nopixel_value_i = np.sum(img == i)
    hist[i] = nopixel_value_i
  return hist

**Option 1 - Single-image Cumulative Histogram:** Compute the Cumulative Histogram $hc(l_i)$ for each image $l_i$ ∈ $L$ and use it as a transform function to equalize the histogram of each image.

In [ ]:
def single_histogram(img):
  no_levels = 256 # 256 is the intesity level of a gray scale image
  hist = histogram(img, no_levels) # histogram
  histC = np.zeros(no_levels).astype(int) # cumulative histogram

  histC[0] = hist[0]
  for i in range(1, no_levels):
    histC[i] = hist[i] + histC[i-1]

  hist_transf = np.zeros(no_levels).astype(np.uint8)
  N,M = img.shape
  img_eq = np.zeros([N,M]).astype(np.uint8)

  for z in range(no_levels):
    s = ((no_levels-1)/float(M*N))*histC[z]
    hist_transf[z] = s
    img_eq[np.where(img==z)] = s

  return img_eq

**Option 2 - Joint Cumulative Histogram:** Compute a single Cumulative Histogram $hc(L)$ over all images in $L$ and use it as a transform function to equalize each image

In [ ]:
def joint_histogram(imglow):
  no_levels = 256
  N, M = imglow[0].shape
  hist = np.zeros(no_levels).astype(int)
  histC = np.zeros(no_levels).astype(int)

  for j in range(0,4):
    for i in range(no_levels):
      nopixel_value_i = np.sum(imglow[j] == i)
      hist[i] = nopixel_value_i

  histC[0] = hist[0]
  for i in range(1, no_levels):
    histC[i] = hist[i] + histC[i-1]

  hist_transf = np.zeros(no_levels).astype(np.uint8)
  img_eq = []
  img_eq.append(np.zeros([N,M]).astype(np.uint8))
  img_eq.append(np.zeros([N,M]).astype(np.uint8))
  img_eq.append(np.zeros([N,M]).astype(np.uint8))
  img_eq.append(np.zeros([N,M]).astype(np.uint8))

  for i in range(0,4):
    for z in range(no_levels):
      s = ((no_levels-1)/float(M*N))*histC[z]
      hist_transf[z] = s
      img_eq[i][np.where(imglow[i]==z)] = s

  return img_eq


### **2.2 Gamma Correction**

**Option 3 - Gamma Correction:** Implement the pixel-wise enhancement function called Gamma Correction, using the following:

$\hat{L_i}(x,y) =  \lfloor 255·((L_i(x,y)/255.0)^{1/γ}) \rfloor ,$

where $\hat{L_i}$ is the resulting image and γ is a parameter input by the user.

In [ ]:
def gamma_correction(li, gama):
  L_hat = li/255
  L_hat = L_hat ** (1/float(gama))
  L_hat = L_hat*255

  return L_hat

## **3. Superresolution**

Let’s assume that each of the low resolution images $L$ is a different “view” of the exact same scene. We can use those images (post enhancement) to compose a higher resolution version $\hat{H}$ (to simplify our task, this higher resolution will always be double the original). We propose a very simple composition method, as in the example below:

$l_1 = \begin{bmatrix} 100 & 101\\ 110 & 111\end{bmatrix},$ $l_2 = \begin{bmatrix} 200 & 201\\ 210 & 211\end{bmatrix},$ $l_3 = \begin{bmatrix} 300 & 301\\ 310 & 311\end{bmatrix},$ $l_4 = \begin{bmatrix} 400 & 401\\ 410 & 411\end{bmatrix}$
$$
$$
$$
$$
$\hat{H} = \begin{bmatrix} 100 & 300 & 101 & 301\\ 200 & 400 & 201 & 401\\ 110 & 310 & 111 & 311\\ 210 & 410 & 211 & 411\end{bmatrix}$

In [ ]:
def superresolution(l1, l2, l3, l4):
    N, M = l1.shape
    H_hat = np.zeros((N*2, M*2))

    H_hat[::2, ::2] = l1
    H_hat[::2, 1::2] = l3
    H_hat[1::2, ::2] = l2
    H_hat[1::2, 1::2] = l4

    return H_hat

## **4.Comparing against reference**

The program will compare the enhanced image $\hat{H}$ against the reference image $H$. This comparison must use the root mean squared error (RMSE). We will print this error in the screen, rounding to 4 decimal places.

$L_{RMSE}(H,\hat{H})=\sqrt{\dfrac{\sum{}_i\sum{}_j((H(i,j)-\hat{H}(i,j))^2}{N·M}}  $

In [ ]:
def rmse(H, H_hat):
  rmse = 0

  squared_dif = (H - H_hat) ** 2
  mse = np.mean(squared_dif)
  rmse = np.sqrt(mse)

  print(round(rmse,4))

## **Output**

In [ ]:
match int(f):
  case 0: # No enhancement
    h_hat = superresolution(imglow[0], imglow[1], imglow[2], imglow[3])
    rmse(imghigh, h_hat)
  case 1: # Single image cumulative histogram
    img0_eq = single_histogram(imglow[0])
    img1_eq = single_histogram(imglow[1])
    img2_eq = single_histogram(imglow[2])
    img3_eq = single_histogram(imglow[3])
    h_hat = superresolution(img0_eq, img1_eq, img2_eq, img3_eq)
    rmse(imghigh, h_hat)
  case 2: # Joint image cumulative histogram
    img_eq = joint_histogram(imglow)
    h_hat = superresolution(img_eq[0], img_eq[1], img_eq[2], img_eq[3])
    rmse(imghigh, h_hat)
  case 3: # Gamma correction
    img0_eq = gamma_correction(imglow[0],gama)
    img1_eq = gamma_correction(imglow[1],gama)
    img2_eq = gamma_correction(imglow[2],gama)
    img3_eq = gamma_correction(imglow[3],gama)
    h_hat = superresolution(img0_eq, img1_eq, img2_eq, img3_eq)
    rmse(imghigh, h_hat)
  case _:
    print("Invalid Method!")

6.89123424366921
